# Design goals
- [ ] 1. Cheap
- [ ] 2. Small
- [ ] 3. Integrated (no external wires, no detachable modules)
- [ ] 4. Safe (no exposed high voltage, no exposed sharp edges)
- [ ] 5. Designed for manufacturing
- [ ] 6. Low power consumption

# Power source
A single 18650 cell.  It should last longer than a child's attention span.  The cell will not be charged in the car, but removed and charged in a dedicated charger.

# Battery protection
The battery will be protected by a DW01A and FS8205A MOSFETs.  Up to three FS8205A can be paralleled to increase the current rating.
These components can be salvaged from AliExpress modules.

The MOSFETs will serve dual use as a power switch.  By using a PMOS to cut the power to the DW01A, it will be tricked into thinking that the battery is empty, and it will disconnect the battery from the load.

To turn the car on, the user will press a momentary switch connected to the 2pin-header.  This will turn on the PMOS, which will power the DW01A.  This will turn on the FS8205A, which will connect the battery to the load.  Once the MCU is running, it should turn on Q3, so that the user can release the button and the car will stay on.

To turn the car off, the user pushes the switch again.  This will pull down the GPIO of the MCU.  The MCU will turn off Q3, which will cut the power to the DW01A.  The DW01A will disconnect the battery from the load.

# Boost converter
The XR2981 and its inductor can be salvaged from cheap AliExpress modules.  It can be used to boost the battery voltage to 8.4V for the steering motor.  It should easily be capable of 2A.  The inductor might need to be replaced with a 3.3uH one.

[Design procedure](https://github.com/LieBtrau/electronics-design-data/blob/master/circuits/boost-converter/boost-converter.ipynb),
based on [Jurgen Hubner : Boost Converter: Design, Circuit, Equations & More](https://www.powerelectronicsnews.com/the-dc-dc-boost-converter-power-supply-design-tutorial-section-5-1/)

# 3V3 regulator
The 3V3 regulator will be a XC6220B331MR-G.  It's easily capable of delivering the 300mA current peaks of the ESP32.  It's low quiescent current is of no concern, because the power to the battery will be cut when the car is not in use.

# MCU
ESP-C3 would have been nice, but there's no support for Bluetooth-SPP, so the ESP32 it is.

# Motor driver
The cheapest option is to build a H-bridge with discrete MOSFETs.  [EDN](https://www.edn.com/microcontroller-drives-piezoelectric-buzzer-at-high-voltage/) features a simple, high-current, non-protected H-bridge.
MOSFETS are needed because the steering motor requires 0.5A@3.7V.  For driving 1A@8.4V will be needed.

## Current sense amplifier
As 8.4V for the motor is actually too high, we might burn the little thing quickly, the motor current will be monitored.
* The amplification is 2x
* Low pass filter is > 20kHz
* A diode is used to build a precision rectifier, followed by an averaging filter (R||C).  This should make it less critical to time the ADC-sampling.

# Firmware
* Ramp up PWM for the thrust motor slowly to avoid a current spike.